In [1]:
from bs4 import BeautifulSoup
import requests
import urllib3
from pymongo import MongoClient
import re
from multiprocessing import Pool
import traceback
from selenium import webdriver
import time
import pandas as pd

/home/senior/.conda/envs/python38/lib/python3.8/site-packages/requests/__init__.py:78: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({0}) or chardet ({1}) doesn't match a supported "


In [2]:
def handle_error(e):
    #處理 child process 的錯誤，不然 code 寫錯時，不會回報任何錯誤
    traceback.print_exception(type(e), e, e.__traceback__)

In [3]:
def patent_soup(patent_num):
    urllib3.disable_warnings() #禁用各種警告
    response = requests.get('https://patents.google.com/patent/' + str(patent_num),verify = False)
    response.encoding='utf-8'
    if response.status_code != 200:
        pass
    else:
        soup =  BeautifulSoup(response.text, 'html.parser') 
        try:
            appDate = soup.find('time', itemprop="filingDate").string.replace('-', '')
            
            # Classifications
            try:
                classifications = soup.find_all('span', itemprop="Code")
                classification =[]
                for i in range(0,len(classifications)):
                    try:
                        if len(classifications[i].string)>len(classifications[i+1].string):
                            classification.append(classifications[i].string)
                        elif i == len(classifications)-2:
                            classification.append(classifications[i+1].string) 
                    except:
                        pass
                if classification[0] == '':
                    classification = None
            except:
                classification = None
        except:
            return None
        
    return appDate, classification

In [1]:
# 取得html
def soup(url):
    urllib3.disable_warnings() #禁用各種警告
    response = requests.get(url,verify = False)
    response.encoding='utf-8'
    if response.status_code != 200:
        pass
    else:
        soup =  BeautifulSoup(response.text, 'html.parser') 
        try:
            patent_num = soup.find('dd',itemprop='publicationNumber').string
            appDate = soup.find('time', itemprop="filingDate").string.replace('-', '')
            pubDate = soup.find('time', itemprop="publicationDate").string.replace('-', '')
            abstract = soup.find('div',{"class":"abstract"}).string
            title = re.sub('\s','',soup.find('span',itemprop="title").string)
            
            #current assignee
            try:
                current = soup.find_all('dd' ,itemprop="assigneeCurrent")
                current_assignee = []
                for i in current:
                    i = re.sub('\n','',i.string)
                    i = ' '.join(i.split())
                    current_assignee.append(i)
            except:
                current_assignee = None
            
            #original assignee
            try:
                original = soup.find_all('dd' ,itemprop="assigneeOriginal")
                original_assignee = []
                for i in original:
                    original_assignee.append(i.string)
            except:
                original_assignee = None
            
            #graDate
            Date = soup.find_all('tr',itemprop="legalEvents")
            graDate = 'N/A'
            for i in Date:
#                 if re.search('.*Grant.*',i.text) != None or re.search('.*grant.*',i.text) != None:
#                     graDate =re.sub('\s','',i.text)[:10].replace('-', '')
#                 else:
#                     graDate = graDate
                if i.find('td', itemprop='title').text == 'Information on status: patent grant':
                    graDate =re.sub('\s','',i.text)[:10].replace('-', '')
            
            # inventorName
            try:
                inventorNames = soup.find_all('dd',itemprop="inventor")
                inventorName = []
                for i in inventorNames:
                    inventorName.append(i.string)
            except:
                inventorName = None

            # description
#             try:
#                 description = soup.find_all('p')
#                 description_list = []
#                 for i in description:
#                     description_list.append(re.sub('\s','',i.text))
#                 description = ("".join(description_list)) 
#             except:
#                 description = None
            if graDate != 'N/A':
                try:
                    descriptions = soup.find_all('div', class_="description-paragraph")
                    description = ''
                    for i in descriptions:
                        description += i.text
                except:
                    description = None
            else:
                try:
                    descriptions = soup.find_all('div', class_="description-line")
                    description = ''
                    for i in descriptions:
                        description += i.text
                except:
                    description = None
                                
            # detailed_description
#             try:
#                 detailed_description_list = soup.find_all('p')
#                 detailed_description_start = 0
#                 detailed_description = ''
                            
#                 for num,i in enumerate(detailed_description_list):
#                     if re.sub('\s','',i.text) == '具体实施方式':
#                         detailed_description_start = num
#                 if detailed_description_start > 0:
#                     for i in range(detailed_description_start,len(detailed_description_list)):
#                         detailed_description += re.sub('\s','',detailed_description_list[i].text)
#                     detailed_description = "".join(detailed_description)
#                 else:
#                     detailed_description = None
#             except:
#                 detailed_description = None
                
            # claim
            try:
                claims = soup.find_all('div',{'class':'claim-text'})
                claim = []
                j = 0
                for i in claims:
                    if re.sub('\s','',i.text) != '':
                        claim.append(re.sub('\s','',i.text))
            #分類
                for i in range(1,len(claim)-1):
                    try:
                        int(claim[i][0])
                        j = i
                    except:
                        a = str(claim[j])+str(claim[i])
                        claim[j] = a
                        claim[i] = '' 
                for i in claim:
                    if '' in claim:
                        claim.remove('')
            except:
                claim = None

            # patentCited    
            try:
                patentCiteds = soup.find_all('tr', itemprop="backwardReferences")
                patentCited_list = []
                patentCited_assignee_list = []
                patentCited = []
                for i in patentCiteds:
                    patentCited_assignee_list.append(i.find('span', itemprop="assigneeOriginal").text)
                    e = re.sub('\s','',i.text).find("(")
                    patentCited_list.append(re.sub('\s','',i.text)[:e])
                if len(patentCited_list) != 0 :
                    for j in patentCited_list:
                        pclist = {'pto':None,'patentNumber':None,'kindcode':None, 'appDate':None, 'assignee':None, 'classification':None}
                        pclist['pto'] = j[:2]
                        try:
                            int(j[-2])
                            try:
                                int(j[-1])
                                pclist['patentNumber'] = j[2:]
                                pclist['kindcode'] = None
                            except:
                                pclist['patentNumber'] = j[2:-2]
                                pclist['kindcode'] = j[-1]
                        except:
                            pclist['patentNumber'] = j[2:-2]
                            pclist['kindcode'] = j[-2:]
                        try:                   
                            cited_appDate, cited_classification = patent_soup(j)
                            pclist['appDate'] = cited_appDate
                            pclist['classification'] = cited_classification
                        except:
                            pclist['appDate'] = None
                            pclist['classification'] = None
                        patentCited.append(pclist)
                    for i in range(len(patentCited)):
                        try:
                            patentCited[i]['assignee'] = patentCited_assignee_list[i]
                        except:
                            patentCited[i]['assignee'] = None
                else:
                    patentCited = None
                    
            except:
                patentCited = None
                
            # Family To Family Citing      
            try:
                family_to_family_citings = soup.find_all('tr', itemprop="backwardReferencesFamily")
                family_to_family_citing_list = []
                family_to_family_citing_assignee_list = []
                family_to_family_citing = []
                for i in family_to_family_citings:
                    family_to_family_citing_assignee_list.append(i.find('span', itemprop="assigneeOriginal").text)
                    e = re.sub('\s','',i.text).find("(")
                    family_to_family_citing_list.append(re.sub('\s','',i.text)[:e])
                    
                if len(family_to_family_citing_list) != 0 :
                    for j in family_to_family_citing_list:
                        clist = {'pto':None,'patentNumber':None,'kindcode':None, 'appDate':None, 'assignee':None, 'classification':None}
                        clist['pto'] = j[:2]
                        try:
                            int(j[-2])
                            try:
                                int(j[-1])
                                clist['patentNumber'] = j[2:]
                                clist['kindcode'] = None
                            except:
                                clist['patentNumber'] = j[2:-2]
                                clist['kindcode'] = j[-1]
                        except:
                            clist['patentNumber'] = j[2:-2]
                            clist['kindcode'] = j[-2:]
                        try:                   
                            citing_appDate, citing_classification = patent_soup(j)
                            clist['appDate'] = citing_appDate
                            clist['classification'] = citing_classification
                        except:
                            clist['appDate'] = None
                            clist['classification'] = None
                        family_to_family_citing.append(clist) 
                    for i in range(len(family_to_family_citing)):
                        try:
                            family_to_family_citing[i]['assignee'] = family_to_family_citing_assignee_list[i]
                        except:
                            family_to_family_citing[i]['assignee'] = None
                else:
                    family_to_family_citing = None
            except:
                family_to_family_citing = None
                
            # ＣountryStatus
            try:
                status =  soup.find_all('tr', itemprop="countryStatus")
                countrystatus = []
                for i in status:
                    countrycode = i.find('span', itemprop="countryCode")
                    try:
                        countrystatus.index(countrycode.string)
                    except:
                        countrystatus.append(countrycode.string)
            except:
                countrystatus = None
            
            # Cited       
            try:
                citeds_1 = soup.find_all('tr' ,itemprop="forwardReferencesOrig" )
                #citeds_2 = soup.find_all('tr', itemprop="forwardReferencesFamily")
                cited_list = []
                cited_assignee_list = []
                cited = []
                for i in citeds_1:
                    cited_assignee_list.append(i.find('span', itemprop="assigneeOriginal").text)
                    e = re.sub('\s','',i.text).find("(")
                    cited_list.append(re.sub('\s','',i.text)[:e])
#                 for i in citeds_2:
#                     e = re.sub('\s','',i.text).find("(")
#                     cited_list.append(re.sub('\s','',i.text)[:e])
                    
                if len(cited_list) != 0 :
                    for j in cited_list:
                        clist = {'pto':None,'patentNumber':None,'kindcode':None, 'appDate':None, 'assignee':None, 'classification':None}
                        clist['pto'] = j[:2]
                        try:
                            int(j[-2])
                            try:
                                int(j[-1])
                                clist['patentNumber'] = j[2:]
                                clist['kindcode'] = None
                            except:
                                clist['patentNumber'] = j[2:-2]
                                clist['kindcode'] = j[-1]
                        except:
                            clist['patentNumber'] = j[2:-2]
                            clist['kindcode'] = j[-2:]
                        try:                   
                            cited_appDate, cited_classification = patent_soup(j)
                            clist['appDate'] = cited_appDate
                            clist['classification'] = cited_classification
                        except:
                            clist['appDate'] = None
                            clist['classification'] = None
                        cited.append(clist)
                    for i in range(len(cited)):
                        try:
                            cited[i]['assignee'] = cited_assignee_list[i]
                        except:
                            cited[i]['assignee'] = None
                
                else:
                    cited = None
            except:
                cited = None
                
            # Family To Family Cited      
            try:
                family_to_family_citeds = soup.find_all('tr', itemprop="forwardReferencesFamily")
                family_to_family_cited_list = []
                family_to_family_cited_assignee_list = []
                family_to_family_cited = []
                for i in family_to_family_citeds:
                    family_to_family_cited_assignee_list.append(i.find('span', itemprop="assigneeOriginal").text)
                    e = re.sub('\s','',i.text).find("(")
                    family_to_family_cited_list.append(re.sub('\s','',i.text)[:e])
                    
                if len(family_to_family_cited_list) != 0 :
                    for j in family_to_family_cited_list:
                        clist = {'pto':None,'patentNumber':None,'kindcode':None, 'appDate':None, 'assignee':None, 'classification':None}
                        clist['pto'] = j[:2]
                        try:
                            int(j[-2])
                            try:
                                int(j[-1])
                                clist['patentNumber'] = j[2:]
                                clist['kindcode'] = None
                            except:
                                clist['patentNumber'] = j[2:-2]
                                clist['kindcode'] = j[-1]
                        except:
                            clist['patentNumber'] = j[2:-2]
                            clist['kindcode'] = j[-2:]
                        try:                   
                            cited_appDate, cited_classification = patent_soup(j)
                            clist['appDate'] = cited_appDate
                            clist['classification'] = cited_classification
                        except:
                            clist['appDate'] = None
                            clist['classification'] = None
                        family_to_family_cited.append(clist) 
                    for i in range(len(family_to_family_cited)):
                        try:
                            family_to_family_cited[i]['assignee'] = family_to_family_cited_assignee_list[i]
                        except:
                            family_to_family_cited[i]['assignee'] = None
                else:
                    family_to_family_cited = None
            except:
                family_to_family_cited = None

            # Classifications
            try:
                classifications = soup.find_all('span', itemprop="Code")
                classification =[]
                for i in range(0,len(classifications)):
                    try:
                        if len(classifications[i].string)>len(classifications[i+1].string):
                            classification.append(classifications[i].string)
                        elif i == len(classifications)-2:
                            classification.append(classifications[i+1].string) 
                    except:
                        pass
                if classification[0] == '':
                    classification = None
            except:
                classification = None
                
            # Similar Documents
            try:
                similar_docs = soup.find_all('tr', itemprop="similarDocuments")
                similarDocuments = []
                similar_doc =[]
                doc_title = []
                doc_publication = []
                doc_time = []

                for doc in similar_docs:
                    publication = doc.find_all('span')
                    time = doc.find_all('time')
                    titles = doc.find_all('td', itemprop="title")

                    for j1 in publication:
                        if len(j1.text.strip())!=2:
                            doc_publication.append(j1.text.strip())
                    for j2 in time:
                        doc_time.append(j2.text.strip())
                    for j3 in titles:
                        doc_title.append(j3.text.strip())
                similarDocuments = []
                for i in range(len(doc_publication)):
                    doc={
                        'publication' : doc_publication[i],
                        'time' : doc_time[i],
                        'title' : doc_title[i]
                        }
                    similarDocuments.append(doc)
            except:
                similarDocuments = None
                

            data = {
            'patentNumber':patent_num,
            'inventorName':inventorName,
            'current_assignee':current_assignee,
            'original_assignee':original_assignee,
            'appDate':appDate,
            'pubDate':pubDate ,
            'graDate':graDate,
            'classification':classification,
            'brief':abstract,
            'title':title,
            'description':description,
#             'detailed_description':detailed_description,
            'claim':claim,
            'countrystatus':countrystatus,
            'patentCitation':patentCited,
            'family_to_family_citing':family_to_family_citing,
            'cited':cited,
            'family_to_family_cited':family_to_family_cited,
            'similarDocuments':similarDocuments
            }
            return data
        except:
            return None
        
def search_google_patent(keyword='', assignee=[], s_date='', e_date=''):
    assignees = ''
    for i in assignee:
        for j in i.split():
            assignees += str(j)
            assignees += '+'
        assignees += ','
    url = 'https://patents.google.com/?q=' + keyword + '&assignee=' + assignees + '&country=US&before=filing:' + e_date + '&after=filing:' + s_date + '&language=ENGLISH&type=PATENT'
    
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--disable-infobars')
    options.add_argument('--no-sandbox')

    driver = webdriver.Chrome(chrome_options=options, executable_path='./chromedriver')
    driver.get(url)
    time.sleep(2)
    html = driver.page_source
    driver.close()
    soup = BeautifulSoup(html, "html.parser")
    
    csv_url = 'https://patents.google.com/' + soup.find('a', class_='style-scope search-results')['href']
    urllib.request.urlretrieve(csv_url, "US_patent.csv")
    

In [2]:
from pymongo import MongoClient
from sshtunnel import SSHTunnelForwarder
import pymongo
from tqdm import tqdm

def insert_data(year, data):
    MONGO_HOST = "140.127.32.24"
    MONGO_USER = "chiayu"
    MONGO_PASS = "npustmiscylab7097"
    MONGO_DB = "2022_Patent_Innovation"
    MONGO_COLLECTION = "20" + year + "_Patent_Innovation"
    
    server = SSHTunnelForwarder(
        MONGO_HOST,
        ssh_username=MONGO_USER,
        ssh_password=MONGO_PASS,
        remote_bind_address=('140.127.32.23', 27017)
    )
    
    server.start()

    conn = pymongo.MongoClient('127.0.0.1',server.local_bind_port)
    conn.admin.authenticate('dnlab', 'dnlab8787')
    conn_db = conn[MONGO_DB]
    conn_db_collection = conn_db[MONGO_COLLECTION]
    
    conn_db_collection.insert_one(data)

    server.stop()
    
def SearchYear(year):
    MONGO_HOST = "140.127.32.24"
    MONGO_USER = "chiayu"
    MONGO_PASS = "npustmiscylab7097"
    MONGO_DB = "Google_Patent_Research"
    MONGO_COLLECTION = "application_20" + year
    
    server = SSHTunnelForwarder(
        MONGO_HOST,
        ssh_username=MONGO_USER,
        ssh_password=MONGO_PASS,
        remote_bind_address=('140.127.32.23', 27017)
    )
    
    server.start()

    conn = pymongo.MongoClient('127.0.0.1',server.local_bind_port)
    conn.admin.authenticate('dnlab', 'dnlab8787')
    conn_db = conn[MONGO_DB]
    conn_db_collection = conn_db[MONGO_COLLECTION]
    
    #建立索引
    #conn_db_collection.create_index("assignees.orgname", name='orgname', background=True)
    #刪除索引
    #data = conn_db_collection.find_one({"assignees.orgname":'Apple Inc'})
    #data = conn_db_collection.find({"assignees.orgname":'Apple Inc'}).explain()["executionStats"]
    data = list(conn_db_collection.find({}, {'_id':0, 'url':1}))
    #information = conn_db_collection.index_information()
    #information = conn_db_collection.index_information()

    server.stop()
    
    return data

def SearchPatentNumber(year, num):
    MONGO_HOST = "140.127.32.24"
    MONGO_USER = "chiayu"
    MONGO_PASS = "npustmiscylab7097"
    MONGO_DB = "2022_Patent_Innovation"
    MONGO_COLLECTION = "20" + year + "_Patent_Innovation"
    
    server = SSHTunnelForwarder(
        MONGO_HOST,
        ssh_username=MONGO_USER,
        ssh_password=MONGO_PASS,
        remote_bind_address=('140.127.32.23', 27017)
    )
    
    server.start()

    conn = pymongo.MongoClient('127.0.0.1',server.local_bind_port)
    conn.admin.authenticate('dnlab', 'dnlab8787')
    conn_db = conn[MONGO_DB]
    conn_db_collection = conn_db[MONGO_COLLECTION]
    
    conn_db_collection.create_index("patentNumber", background=True)
    data = conn_db_collection.find_one({"patentNumber":num},{'_id':0, 'patentNumber':1})

    server.stop()
    
    return data

ModuleNotFoundError: No module named 'sshtunnel'

In [148]:
#搜尋patent csv
keyword = ''
assignee = ['Apple Inc.','Google LLC.'] #ex.['Apple Inc.','Google LLC.']
s_date = '20150101'
e_date = '20201231'
search_google_patent(keyword, assignee, s_date, e_date)

'https://patents.google.com/?q=&assignee=Apple+Inc.+,Google+LLC.+,&country=US&before=filing:20201231&after=filing:20150101&language=ENGLISH&type=PATENT'

In [8]:
#爬蟲並新增資料
data = pd.read_csv('US.csv', encoding= 'unicode_escape')
urls = list(data.iloc[:, 0])
del urls[0]

for i in tqdm(range(0,10)):
    data = soup(urls[i])
    insert_data(data)

100%|███████████████████████████████████████████████████████████████████████████████████| 10/10 [05:29<00:00, 32.96s/it]


In [ ]:
#資料庫查patent_url抓資料
n_num = 1
for i in range(10, 21):
    patent = SearchYear(str(i))
    print(str(n_num) + "/" + '11')
    for j in tqdm(patent):
        patent_num = j['url'].replace('https://patents.google.com/patent/', '')
        Is_patent_num = SearchPatentNumber(str(i), str(patent_num))
        if Is_patent_num:
            n_num += 1
            continue
        else:
            data = soup(j['url'])
            insert_data(str(i), data)
    n_num += 1

1/11


  0%|                                                                            | 15/333209 [00:07<44:22:02,  2.09it/s]